In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [13]:
## first retrieval tool -- wikipedia
wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
tool_wiki=WikipediaQueryRun(api_wrapper=wikipedia_wrapper)

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

## second retrieval tool --  a website
loader = WebBaseLoader("https://www.britannica.com/place/Ottoman-Empire")
documents_loaded = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200).split_documents(documents_loaded)

embedding = HuggingFaceEmbeddings(model_name = "thenlper/gte-small")
db = FAISS.from_documents(documents, embedding)

retriever = db.as_retriever()

c:\Users\User\miniconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
from langchain.tools.retriever import create_retriever_tool
tool_web = create_retriever_tool(retriever, "brief_ottoman_empire_history",
                        "Search for information about Ottoman Empire. For any questions about Ottoman Empire, use this tool")
tool_web.name

'brief_ottoman_empire_history'

In [15]:
## third retrieval tool -- arxiv
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
tool_arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
tool_arxiv.name

'arxiv'

In [16]:
## combine tools
tools = [tool_wiki, tool_web, tool_arxiv]

In [26]:
from langchain_community.chat_models import ChatOllama
llm=ChatOllama(model="llama2")

In [20]:
from langchain import hub
# Get the prompt to use 
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [27]:
## agent
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

## agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [28]:
agent_executor.invoke({"input": "Tell me about Ottoman Empire briefly."})



> Entering new AgentExecutor chain...

Of course, I'd be happy to help! The Ottoman Empire, also known as the Turkish Empire or Osmanlı İmparatorluğu in Turkish, was a vast and influential empire that existed from the 13th to the 20th century. It was one of the largest and most powerful empires in history, spanning across three continents and leaving a lasting legacy in the modern world.

Here are some key facts about the Ottoman Empire:

1. Establishment: The Ottoman Empire was established in the late 13th century by Osman Bey, a Turkish tribal leader. It began as a small tribe in northwestern Anatolia (modern-day Turkey) and gradually expanded through military conquests and diplomatic alliances.
2. Expansion: The Ottoman Empire grew rapidly through the 14th to 16th centuries, conquering much of the Middle East, North Africa, and Eastern Europe. Its territories included modern-day Turkey, Syria, Lebanon, Jordan, Israel, Palestine, Egypt, Libya, Tunisia, Algeria, and parts of Iraq, K

{'input': 'Tell me about Ottoman Empire briefly.',
 'output': "\nOf course, I'd be happy to help! The Ottoman Empire, also known as the Turkish Empire or Osmanlı İmparatorluğu in Turkish, was a vast and influential empire that existed from the 13th to the 20th century. It was one of the largest and most powerful empires in history, spanning across three continents and leaving a lasting legacy in the modern world.\n\nHere are some key facts about the Ottoman Empire:\n\n1. Establishment: The Ottoman Empire was established in the late 13th century by Osman Bey, a Turkish tribal leader. It began as a small tribe in northwestern Anatolia (modern-day Turkey) and gradually expanded through military conquests and diplomatic alliances.\n2. Expansion: The Ottoman Empire grew rapidly through the 14th to 16th centuries, conquering much of the Middle East, North Africa, and Eastern Europe. Its territories included modern-day Turkey, Syria, Lebanon, Jordan, Israel, Palestine, Egypt, Libya, Tunisia, 

In [31]:
## ask about a paper in arxiv
agent_executor.invoke({"input": "what is the paper titled as Greedy equivalence search for nonparametric graphical models about?"}) 



> Entering new AgentExecutor chain...

The paper "Greedy Equivalence Search for Nonparametric Graphical Models" is about developing an efficient algorithm for searching for equivalent probabilistic graphical models, also known as Bayesian networks or Markov random fields. The authors propose a greedy search algorithm that starts with an initial model and iteratively adds, deletes, and modifies edges to find the most parsimonious (i.e., simplest) model that is equivalent to the original model.

The algorithm works by starting with the given graphical model and then applying a set of heuristics to guide the search towards simpler models. The heuristics are based on the idea of minimizing the number of non-negligible probabilities in the model, which corresponds to removing unnecessary edges and nodes while preserving the structural equation.

The authors demonstrate the effectiveness of their algorithm through experiments on several datasets, showing that it can find significantly simp

{'input': 'what is the paper titled as Greedy equivalence search for nonparametric graphical models about?',
 'output': '\nThe paper "Greedy Equivalence Search for Nonparametric Graphical Models" is about developing an efficient algorithm for searching for equivalent probabilistic graphical models, also known as Bayesian networks or Markov random fields. The authors propose a greedy search algorithm that starts with an initial model and iteratively adds, deletes, and modifies edges to find the most parsimonious (i.e., simplest) model that is equivalent to the original model.\n\nThe algorithm works by starting with the given graphical model and then applying a set of heuristics to guide the search towards simpler models. The heuristics are based on the idea of minimizing the number of non-negligible probabilities in the model, which corresponds to removing unnecessary edges and nodes while preserving the structural equation.\n\nThe authors demonstrate the effectiveness of their algorith